In [ ]:
from petastorm import make_reader
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis,skew
from scipy.signal import periodogram
import scipy.signal
from ecgdetectors import Detectors
import matplotlib.ticker as ticker
import pywt
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from matplotlib.widgets import TextBox, Button
import sys
import pandas as pd
import seaborn as sn
import os
from math import fsum
sys.path.append(os.path.join(os.getcwd(), ".."))
from shared_utils import Time_series_dimensions_calculus as TSD
path_formatted_glasgow = "/workspaces/maitrise/data/20221006_physio_quality/set-a/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"
path_csv_ref_label = "/workspaces/maitrise/data/20221006_physio_quality/set-a/REFERENCE.csv"

In [ ]:
with make_reader(path_petastorm) as reader:
    for sample in reader:
        data = sample
        if data.signal_quality == "acceptable".encode():
            break
        else : 
            pass

print(data)
print(data.signal_quality)
ECG_signal = data.signal
ECG_lead = data.signal_names
fs = data.sampling_frequency

dico_ECG = {}

for i,j in zip(ECG_lead,range(12)):
     dico_ECG[i] = ECG_signal[:,j]

In [ ]:
def add_observational_noise(sig,SNR):
    Power_sig = np.mean(np.abs(sig)**2)
    P_db = 10*np.log10(Power_sig)
    noisedb = P_db - SNR
    sd_db_watts = 10**(noisedb/10)
    #sd_noise = np.sqrt(Power_sig/(SNR))
    noise = np.random.normal(0,np.sqrt(sd_db_watts),len(sig))
    sig_noisy = sig+noise
    return sig_noisy

In [ ]:
SNR_level = np.linspace(-10,100,50)

def TSD_mean_calculator(signal,segment_length = 100,dt = 0.001):
    w = 1
    Ds = np.array([])
    while (w*segment_length)<=len(signal):
        sig_c  = signal[int((w-1)*segment_length):int((w)*segment_length)]
        L1 = TSD.Lq_k(sig_c,1,1/dt)
        L2 = TSD.Lq_k(sig_c,2,1/dt)
        Dv = (np.log(L1)-np.log(L2))/(np.log(2))   
        Ds = np.append(Ds,Dv)
        w+=1
    return np.mean(Ds),np.std(Ds)

def TSDvsNoiseLevel_array(noise_level,dico_signal,name_lead,fs):
    Dmean = {}
    SD_D = {}
    the_mean_lead_calculator = np.array([])
    the_SDmean_lead_calculator = np.array([])
    
    for name in name_lead:
        Dmean[name.decode('utf8')] = np.array([])
        SD_D[name.decode('utf8')] = np.array([])
    for i in noise_level:
        inter_mean  = np.array([])
        inter_SD  = np.array([])
        for name in name_lead:
            
            Obs = dico_signal[name]
            noise_obs = add_observational_noise(Obs.copy(),i)
            Mean_TSD,SD_TSD = TSD_mean_calculator(noise_obs,TSD.Interval_calculator_lead(Obs.copy(),fs),1/fs)
            inter_mean = np.append(inter_mean,Mean_TSD)
            inter_SD = np.append(inter_SD,SD_TSD)
            Dmean[name.decode('utf8')] = np.append(Dmean[name.decode('utf8')],Mean_TSD)
            SD_D[name.decode('utf8')] = np.append(SD_D[name.decode('utf8')],SD_TSD)
        the_mean_lead_calculator = np.append(the_mean_lead_calculator,np.mean(inter_mean))
        the_SDmean_lead_calculator = np.append(the_SDmean_lead_calculator,np.mean(inter_SD))

    return Dmean,SD_D,the_mean_lead_calculator,the_SDmean_lead_calculator


def plt_TSDvsNoise(noise_lev,dico_sig,name_l,fs):
    Great_mean,Great_SD,mean_TSD_ECG,SD_TSD_ECG= TSDvsNoiseLevel_array(noise_lev,dico_sig,name_l,fs)
    plt.figure()
    plt.plot(noise_lev,mean_TSD_ECG,"ob")
    plt.errorbar(noise_lev,mean_TSD_ECG,SD_TSD_ECG)
    plt.xlabel("SNR (db)")
    plt.ylabel("mean TSD value")
    plt.title(f"TSD vs SNR (db) for average TSD value of all lead") 
    plt.grid()
    plt.show()


    labels = []
    plt.figure()
    colormap = plt.cm.gist_ncar
    plt.gca().set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(0, 1, len(ECG_lead)))))
    for i in name_l:
        plt.plot(noise_lev,Great_mean[i.decode('utf8')])
        labels.append(i.decode('utf8'))
    plt.legend(labels, ncol=4, loc='best', 
           columnspacing=1.0, labelspacing=0.0,
           handletextpad=0.0, handlelength=1.5,
           fancybox=True, shadow=True)
    plt.title("Mean TSD value evolution with SNR (db) for a set of acceptable lead")
    plt.xlabel("SNR (db)")
    plt.ylabel("mean TSD value")
    #plt.ylim([1.9,2.1])
    plt.grid()
    plt.show()

plt_TSDvsNoise(SNR_level,dico_ECG,ECG_lead,fs)